In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import torch
import torchvision
import torchprof
model = torchvision.models.alexnet(pretrained=False).cuda()
x = torch.rand([1, 3, 224, 224]).cuda()
with torchprof.profile(model, use_cuda=True) as prof:
    _ = model(x)



Adding traces:   0%|          | 0/24 [00:00<?, ?it/s]

Adding traces: 100%|██████████| 24/24 [00:00<00:00, 101577.49it/s]


Removing traces:   0%|          | 0/24 [00:00<?, ?it/s]

Removing traces: 100%|██████████| 24/24 [00:00<00:00, 101782.91it/s]

In [3]:
prof.display(min_pct=0)

Make Events:   0%|          | 0/351 [00:00<?, ?it/s]

Make Events: 100%|██████████| 351/351 [00:00<00:00, 62431.65it/s]


Populate ancestors:   0%|          | 0/351 [00:00<?, ?it/s]

Populate ancestors: 100%|██████████| 351/351 [00:00<00:00, 109449.16it/s]


Group:   0%|          | 0/351 [00:00<?, ?it/s]

...


Aggregate:   0%|          | 0/204 [00:00<?, ?it/s]

Aggregate: 100%|██████████| 204/204 [00:00<00:00, 48406.77it/s]


Make rows: 0it [00:00, ?it/s]

Make rows: 2it [00:00, 2852.30it/s]



CPU=14.1ms, CUDA=24.9ms
+-------------+--------------+--------------+-------------+--------------+---------+
| Node        |     Self CPU |          CPU |   Self CUDA |         CUDA |   Count |
|-------------+--------------+--------------+-------------+--------------+---------|
| AlexNet     | 352.8us (3%) | 13.8ms (98%) | 62.5us (0%) | 24.6ms (99%) |       1 |
| aten::zeros |  40.4us (0%) | 313.9us (2%) | 49.2us (0%) | 312.3us (1%) |       1 |
+-------------+--------------+--------------+-------------+--------------+---------+


In [9]:
prof.display(min_pct=1)

Make rows: 0it [00:00, ?it/s]

Make rows: 2it [00:00, 5071.71it/s]



CPU=14.1ms, CUDA=24.9ms
+-------------+--------------+--------------+-------------+--------------+---------+
| Node        |     Self CPU |          CPU |   Self CUDA |         CUDA |   Count |
|-------------+--------------+--------------+-------------+--------------+---------|
| AlexNet     | 352.8us (3%) | 13.8ms (98%) |             | 24.6ms (99%) |       1 |
| aten::zeros |              | 313.9us (2%) |             | 312.3us (1%) |       1 |
+-------------+--------------+--------------+-------------+--------------+---------+


In [5]:
prof.display(min_pct=1, filter=[])

Make rows: 0it [00:00, ?it/s]

Make rows: 204it [00:00, 61112.64it/s]



CPU=14.1ms, CUDA=24.9ms
+----------------------------------------+--------------+--------------+--------------+--------------+---------+
| Node                                   |     Self CPU |          CPU |    Self CUDA |         CUDA |   Count |
|----------------------------------------+--------------+--------------+--------------+--------------+---------|
| AlexNet                                | 352.8us (3%) | 13.8ms (98%) |              | 24.6ms (99%) |       1 |
| ├──classifier                          | 215.2us (2%) |  4.3ms (30%) |              | 12.5ms (50%) |       1 |
| │ ├──1                                 | 269.5us (2%) |  2.5ms (18%) |              |  8.8ms (35%) |       1 |
| │ │ ├──aten::addmm                     |  2.2ms (16%) |  2.2ms (16%) |  8.8ms (35%) |  8.8ms (35%) |       1 |
| │ ├──4                                 |              | 145.8us (1%) |              |   1.9ms (8%) |       1 |
| │ │ ├──aten::addmm                     |              |             

In [7]:
prof.display(sort_by=["self_cuda_time"], min_pct=0, filter=[])

Make rows: 0it [00:00, ?it/s]

Make rows: 204it [00:00, 94181.40it/s]



CPU=14.1ms, CUDA=24.9ms
+----------------------------------------+--------------+--------------+--------------+--------------+---------+
| Node                                   |     Self CPU |          CPU |    Self CUDA |         CUDA |   Count |
|----------------------------------------+--------------+--------------+--------------+--------------+---------|
| AlexNet                                | 352.8us (3%) | 13.8ms (98%) |  62.5us (0%) | 24.6ms (99%) |       1 |
| ├──features                            | 341.4us (2%) |  8.8ms (62%) | 124.5us (0%) | 11.9ms (48%) |       1 |
| │ ├──0                                 | 319.3us (2%) |  4.4ms (31%) | 280.0us (1%) |  4.4ms (17%) |       1 |
| │ │ ├──aten::conv2d                    | 211.2us (1%) |  4.0ms (29%) | 209.7us (1%) |  4.1ms (16%) |       1 |
| │ │ │ └──aten::convolution             |   9.3us (0%) |  3.8ms (27%) |   4.3us (0%) |  3.9ms (16%) |       1 |
| │ │ │   └──aten::_convolution          |  45.7us (0%) |  3.8ms (27%)